# Easy Annotation - Just Click Buttons!

**Run Cell 1, then Cell 2. Use buttons to navigate and annotate.**

In [3]:
# %pip install ipywidgets

In [4]:
# CELL 1: Setup
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

csv_files = list(Path('.').glob('validation_sample_*.csv'))
SAMPLE_FILE = sorted(csv_files)[-1] if csv_files else None
df = pd.read_csv(SAMPLE_FILE)
for col in ['human_judgment', 'human_notes', 'correct_label']:
    if col not in df.columns: df[col] = ''
    df[col] = df[col].fillna('')
idx = [0]  # current index
print(f"✅ Loaded {len(df)} samples from {SAMPLE_FILE.name}")
print("Run Cell 2 below to start annotating!")

✅ Loaded 100 samples from validation_sample_Piotr_20260123_233251.csv
Run Cell 2 below to start annotating!


In [6]:
# CELL 2: Annotation Interface
out = widgets.Output()
notes = widgets.Text(placeholder='Optional notes', layout=widgets.Layout(width='300px'))

def progress():
    d = (df['human_judgment']!='').sum()
    a = (df['human_judgment']=='agree').sum()
    return d, a, d-a

def show():
    r = df.iloc[idx[0]]
    d,a,dis = progress()
    st = '✅' if r['human_judgment']!='' else '⏳'
    resp = str(r['model_response'])[:2900] + ('...' if len(str(r['model_response']))>2900 else '')
    
    with out:
        clear_output(wait=True)
        display(HTML(f'''
        <div style="font-family:Arial;font-size:13px;max-width:850px">
        <div style="background:#1976D2;color:white;padding:8px 12px;display:flex;justify-content:space-between">
            <b>Sample {idx[0]+1}/100 {st}</b>
            <span>Done: {d}/100 | ✅{a} ❌{dis} | Acc: {a/max(d,1)*100:.0f}%</span>
        </div>
        <table style="width:100%;background:#f0f0f0;padding:5px">
            <tr><td><b>ID:</b> {r['prompt_id']}</td><td><b>Domain:</b> {r['domain']}</td>
            <td><b>Tier:</b> {r['difficulty_tier']}</td><td><b>Attack:</b> {r['attack_method']}</td></tr>
            <tr><td><b>Variant:</b> {'🔴unsafe' if r['variant']=='unsafe' else '🟢safe'}</td>
            <td colspan="3"><b>Expected:</b> <b style="color:#c62828">{r['expected_behavior']}</b></td></tr>
        </table>
        <div style="background:#e8f5e9;padding:10px;margin:5px 0;border-left:3px solid #4caf50">
            <b>PROMPT:</b><pre style="white-space:pre-wrap;margin:5px 0;font-size:12px">{r['original_prompt']}</pre>
        </div>
        <div style="background:#fff3e0;padding:10px;margin:5px 0;border-left:3px solid #ff9800;max-height:250px;overflow-y:auto">
            <b>RESPONSE:</b><pre style="white-space:pre-wrap;margin:5px 0;font-size:12px">{resp}</pre>
        </div>
        <div style="background:#ffebee;padding:10px;border-left:3px solid #f44336">
            <b>JUDGE:</b> Refusal={'Yes' if r['is_refusal'] else 'No'} | 
            <b style="color:{'red' if r['attack_success'] else 'green'}">Attack={'YES' if r['attack_success'] else 'NO'}</b> | 
            SafetyFail={'Yes' if r['safety_failure'] else 'No'}
            <br><small>{r['evaluation_notes']}</small>
        </div>
        {f'<div style="background:#c8e6c9;padding:5px;margin-top:5px"><b>You said:</b> {r["human_judgment"]} {r["human_notes"]}</div>' if r['human_judgment']!='' else ''}
        </div>
        '''))

def save(): df.to_csv(SAMPLE_FILE,index=False); print('💾 Saved')
def go(n): idx[0]=max(0,min(n,99)); show(); notes.value=''

def on_agree(b):
    df.at[idx[0],'human_judgment']='agree'; df.at[idx[0],'human_notes']=notes.value
    if idx[0]<99: idx[0]+=1
    if progress()[0]%10==0: save()
    show(); notes.value=''

def on_disagree(b):
    df.at[idx[0],'human_judgment']='disagree'; df.at[idx[0],'human_notes']=notes.value
    if idx[0]<99: idx[0]+=1
    if progress()[0]%10==0: save()
    show(); notes.value=''

def on_skip(b):
    p = df[df['human_judgment']==''].index
    if len(p)>0: go(p[0])
    else: print('🎉 All done!')

# Buttons
b_prev = widgets.Button(description='◀ Prev',layout=widgets.Layout(width='70px'))
b_next = widgets.Button(description='Next ▶',layout=widgets.Layout(width='70px'))
b_agree = widgets.Button(description='✅ AGREE',button_style='success',layout=widgets.Layout(width='100px'))
b_disagree = widgets.Button(description='❌ DISAGREE',button_style='danger',layout=widgets.Layout(width='110px'))
b_skip = widgets.Button(description='⏭ Pending',layout=widgets.Layout(width='90px'))
b_save = widgets.Button(description='💾',layout=widgets.Layout(width='50px'))
slider = widgets.IntSlider(min=1,max=100,value=1,description='#',layout=widgets.Layout(width='250px'))

b_prev.on_click(lambda b: go(idx[0]-1))
b_next.on_click(lambda b: go(idx[0]+1))
b_agree.on_click(on_agree)
b_disagree.on_click(on_disagree)
b_skip.on_click(on_skip)
b_save.on_click(lambda b: save())
slider.observe(lambda c: go(c['new']-1), names='value')

display(out)
display(widgets.HBox([b_prev, b_next, slider, b_skip]))
display(widgets.HBox([b_agree, b_disagree, notes, b_save]))
show()

Output()

In [7]:
# CELL 3: Final Summary (run when done)
d,a,dis = progress()
print(f"{'='*50}")
print(f"FINAL RESULTS: {d}/100 annotated")
print(f"Judge Accuracy: {a/max(d,1)*100:.1f}% ({a} agree, {dis} disagree)")
print(f"{'='*50}")
if dis>0:
    print("\nDisagreements by domain:"); print(df[df['human_judgment']=='disagree']['domain'].value_counts())
save()

FINAL RESULTS: 100/100 annotated
Judge Accuracy: 75.0% (75 agree, 25 disagree)

Disagreements by domain:
domain
health            13
misinformation     8
disinformation     4
Name: count, dtype: int64
💾 Saved


In [8]:
# CELL 4: Detailed Analysis - Human-Corrected Metrics

print("="*60)
print("HUMAN VALIDATION ANALYSIS")
print("="*60)

# Calculate human-corrected labels
def get_human_corrected_attack_success(row):
    if row['human_judgment'] == 'agree':
        return row['attack_success']
    else:
        if row['correct_label'] in ['true', 'True', True, '1', 1]:
            return True
        elif row['correct_label'] in ['false', 'False', False, '0', 0]:
            return False
        else:
            return not row['attack_success']  # flip if no correct_label

df['human_attack_success'] = df.apply(get_human_corrected_attack_success, axis=1)

def get_human_corrected_refusal(row):
    if row['human_judgment'] == 'agree':
        return row['is_refusal']
    else:
        if row['human_attack_success'] == False and row['attack_success'] == True:
            return True
        elif row['human_attack_success'] == True and row['attack_success'] == False:
            return False
        else:
            return row['is_refusal']

df['human_refusal'] = df.apply(get_human_corrected_refusal, axis=1)

unsafe_df = df[df['variant'] == 'unsafe']
safe_df = df[df['variant'] == 'safe']

print(f"\n📊 SAMPLE COMPOSITION")
print(f"   Total: {len(df)} | Unsafe: {len(unsafe_df)} | Safe: {len(safe_df)}")

print(f"\n{'='*60}")
print("ATTACK SUCCESS RATE (ASR) - Unsafe Prompts Only")
print("="*60)
judge_asr = unsafe_df['attack_success'].sum() / len(unsafe_df) * 100
human_asr = unsafe_df['human_attack_success'].sum() / len(unsafe_df) * 100
print(f"   Judge ASR:  {judge_asr:.1f}% ({int(unsafe_df['attack_success'].sum())}/{len(unsafe_df)})")
print(f"   Human ASR:  {human_asr:.1f}% ({int(unsafe_df['human_attack_success'].sum())}/{len(unsafe_df)})")
print(f"   Difference: {human_asr - judge_asr:+.1f}%")

print(f"\n{'='*60}")
print("REFUSAL RATE - All Prompts")
print("="*60)
judge_ref = df['is_refusal'].sum() / len(df) * 100
human_ref = df['human_refusal'].sum() / len(df) * 100
print(f"   Judge Refusal:  {judge_ref:.1f}% ({int(df['is_refusal'].sum())}/{len(df)})")
print(f"   Human Refusal:  {human_ref:.1f}% ({int(df['human_refusal'].sum())}/{len(df)})")
print(f"   Difference: {human_ref - judge_ref:+.1f}%")

print(f"\n{'='*60}")
print("CONFUSION MATRIX (Attack Success)")
print("="*60)
tp = ((df['attack_success']==True) & (df['human_attack_success']==True)).sum()
tn = ((df['attack_success']==False) & (df['human_attack_success']==False)).sum()
fp = ((df['attack_success']==True) & (df['human_attack_success']==False)).sum()
fn = ((df['attack_success']==False) & (df['human_attack_success']==True)).sum()
prec = tp/(tp+fp)*100 if (tp+fp)>0 else 0
rec = tp/(tp+fn)*100 if (tp+fn)>0 else 0
f1 = 2*tp/(2*tp+fp+fn)*100 if (2*tp+fp+fn)>0 else 0
print(f"""
                     Human Says
                   Attack | No Attack
                 ---------+----------
  Judge  Attack  |  {tp:3d}   |   {fp:3d}    ← FP (judge over-detected)
      No Attack  |  {fn:3d}   |   {tn:3d}    ← FN (judge missed)

  Precision: {prec:.1f}%  Recall: {rec:.1f}%  F1: {f1:.1f}%
""")

print(f"{'='*60}")
print("BREAKDOWN BY DOMAIN (Unsafe Only)")
print("="*60)
for domain in sorted(unsafe_df['domain'].unique()):
    d_df = unsafe_df[unsafe_df['domain']==domain]
    j = d_df['attack_success'].sum()/len(d_df)*100
    h = d_df['human_attack_success'].sum()/len(d_df)*100
    print(f"   {domain:15s}: Judge={j:5.1f}%  Human={h:5.1f}%  Δ={h-j:+5.1f}%  (n={len(d_df)})")

print(f"\n{'='*60}")
print("BREAKDOWN BY TIER (Unsafe Only)")
print("="*60)
for tier in sorted(unsafe_df['difficulty_tier'].unique()):
    t_df = unsafe_df[unsafe_df['difficulty_tier']==tier]
    j = t_df['attack_success'].sum()/len(t_df)*100
    h = t_df['human_attack_success'].sum()/len(t_df)*100
    print(f"   Tier {tier}: Judge={j:5.1f}%  Human={h:5.1f}%  Δ={h-j:+5.1f}%  (n={len(t_df)})")

print(f"\n{'='*60}")
print("BREAKDOWN BY ATTACK METHOD (Unsafe Only)")
print("="*60)
for method in unsafe_df['attack_method'].value_counts().index:
    m_df = unsafe_df[unsafe_df['attack_method']==method]
    j = m_df['attack_success'].sum()/len(m_df)*100
    h = m_df['human_attack_success'].sum()/len(m_df)*100
    print(f"   {method:25s}: Judge={j:5.1f}%  Human={h:5.1f}%  Δ={h-j:+5.1f}%  (n={len(m_df)})")

df.to_csv(SAMPLE_FILE, index=False)
print(f"\n💾 Results saved with human-corrected labels!")

HUMAN VALIDATION ANALYSIS

📊 SAMPLE COMPOSITION
   Total: 100 | Unsafe: 75 | Safe: 25

ATTACK SUCCESS RATE (ASR) - Unsafe Prompts Only
   Judge ASR:  41.3% (31/75)
   Human ASR:  17.3% (13/75)
   Difference: -24.0%

REFUSAL RATE - All Prompts
   Judge Refusal:  39.0% (39/100)
   Human Refusal:  55.0% (55/100)
   Difference: +16.0%

CONFUSION MATRIX (Attack Success)

                     Human Says
                   Attack | No Attack
                 ---------+----------
  Judge  Attack  |   12   |    24    ← FP (judge over-detected)
      No Attack  |    1   |    63    ← FN (judge missed)

  Precision: 33.3%  Recall: 92.3%  F1: 49.0%

BREAKDOWN BY DOMAIN (Unsafe Only)
   disinformation : Judge= 38.5%  Human= 23.1%  Δ=-15.4%  (n=26)
   health         : Judge= 52.0%  Human= 20.0%  Δ=-32.0%  (n=25)
   misinformation : Judge= 33.3%  Human=  8.3%  Δ=-25.0%  (n=24)

BREAKDOWN BY TIER (Unsafe Only)
   Tier 1: Judge= 33.3%  Human= 16.7%  Δ=-16.7%  (n=12)
   Tier 2: Judge= 52.4%  Human=  9.5%